In [2]:
pip install -U pandasql

Requirement already up-to-date: pandasql in c:\users\crie072\anaconda3\lib\site-packages (0.7.3)
Note: you may need to restart the kernel to use updated packages.


In [65]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

from pandasql import sqldf

In [4]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20200811


In [5]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

ディレクトリ

In [232]:
data_dir = Path(r"C:\Users\crie072\Documents\21_Online\オンライン入会状況レポート")
data_dir2 = Path(r"C:\Users\crie072\Documents\21_Online")

In [285]:
#input
input_file = "【C116】新規入会者_6月末時点"

input_file2="アフィリエイト広告_累積承認者リスト"

#out_put
output_file = "オンライン入会者の利用状況調査レポート作成用"
output_file2="オンライン入会者の利用状況調査レポート作成用_アフィリエイト別"

In [286]:
df = pd.read_excel(data_dir /r"input"/f"{input_file}.xlsx",encoding='shift-jis')

In [287]:
df.head()

,実績月,入会申込番号,直近６ケ月買物利用金額,直近６ケ月全体利用金額,年令,性別,都道府県名,顧客状況コ－ド,マイペ設定,DM希望店コード,カード券種名,カ－ド加入日,初回利用日,あとからリボ申込有無コード,Weｂ明細有無コード,解約,入会チャネル
0,202006,155051921627812,879,879,39,女性,東京都,0,0,M_上野店,大丸松坂屋VISAゴールド,20200106,20200116,0,1,0,オンライン
1,202006,155051921628414,799,799,27,男性,兵庫県,0,0,Ｄ_神戸店,大丸松坂屋VISAゴールド,20200106,20200118,0,1,0,オンライン
2,202006,155051921636805,150,150,63,男性,大阪府,0,0,Ｄ_梅田店,大丸松坂屋VISAゴールド,20200106,20200115,0,1,0,オンライン
3,202006,155193017345014,2,2,46,男性,茨城県,0,1,Ｄ_東京店,大丸松坂屋VISAゴールド,20200106,20200116,0,1,0,店頭タブレット
4,202006,155193017345790,180,180,59,男性,兵庫県,0,1,Ｄ_梅田店,大丸松坂屋VISAゴールド,20200106,20200209,0,1,0,店頭タブレット


In [288]:
#重複削除
df_dup = df.drop_duplicates(subset=['入会申込番号','実績月'])
df_dup.shape[0]

29580

In [289]:
df_dup.head()

,実績月,入会申込番号,直近６ケ月買物利用金額,直近６ケ月全体利用金額,年令,性別,都道府県名,顧客状況コ－ド,マイペ設定,DM希望店コード,カード券種名,カ－ド加入日,初回利用日,あとからリボ申込有無コード,Weｂ明細有無コード,解約,入会チャネル
0,202006,155051921627812,879,879,39,女性,東京都,0,0,M_上野店,大丸松坂屋VISAゴールド,20200106,20200116,0,1,0,オンライン
1,202006,155051921628414,799,799,27,男性,兵庫県,0,0,Ｄ_神戸店,大丸松坂屋VISAゴールド,20200106,20200118,0,1,0,オンライン
2,202006,155051921636805,150,150,63,男性,大阪府,0,0,Ｄ_梅田店,大丸松坂屋VISAゴールド,20200106,20200115,0,1,0,オンライン
3,202006,155193017345014,2,2,46,男性,茨城県,0,1,Ｄ_東京店,大丸松坂屋VISAゴールド,20200106,20200116,0,1,0,店頭タブレット
4,202006,155193017345790,180,180,59,男性,兵庫県,0,1,Ｄ_梅田店,大丸松坂屋VISAゴールド,20200106,20200209,0,1,0,店頭タブレット


In [290]:
df_dup = df_dup.rename(columns={'実績月':'cp',
                        '入会申込番号':'applino',
                        '直近６ケ月全体利用金額':'credit_amt_6m',
                        '入会チャネル':'channel',
                        '解約':'kaiyaku_f',
                        '初回利用日':'first_use_dt',
                        'カ－ド加入日':'kanyu_dt',
                        'カード券種名':'card'})
df_dup = df_dup.drop(columns={'直近６ケ月買物利用金額','年令','性別','都道府県名','顧客状況コ－ド','マイペ設定','DM希望店コード',
                     'あとからリボ申込有無コード','Weｂ明細有無コード'})
df_dup.head(2)

,cp,applino,credit_amt_6m,card,kanyu_dt,first_use_dt,kaiyaku_f,channel
0,202006,155051921627812,879,大丸松坂屋VISAゴールド,20200106,20200116,0,オンライン
1,202006,155051921628414,799,大丸松坂屋VISAゴールド,20200106,20200118,0,オンライン


In [291]:
print(df_dup.dtypes)

cp                int64
applino           int64
credit_amt_6m     int64
card             object
kanyu_dt          int64
first_use_dt      int64
kaiyaku_f         int64
channel          object
dtype: object


In [292]:
df_dup['cp_dt'] = df_dup['cp']*100+1
df_dup['cp_dt'] = pd.to_datetime(df_dup['cp_dt'], format='%Y%m%d')

In [293]:
df_dup['kanyu_dt']=pd.to_datetime(df_dup['kanyu_dt'], format='%Y%m%d')
df_dup['kanyu_year'] = df_dup['kanyu_dt'].dt.year
df_dup['kanyu_month'] = df_dup['kanyu_dt'].dt.month

df_dup['kanyu_yymm'] = df_dup[['kanyu_year', 'kanyu_month']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)

In [294]:
#経過月数
df_dup['mob']=(df_dup['cp_dt'].dt.year - df_dup['kanyu_dt'].dt.year) * 12 + df_dup['cp_dt'].dt.month - df_dup['kanyu_dt'].dt.month
df_dup.head(2)

,cp,applino,credit_amt_6m,card,kanyu_dt,first_use_dt,kaiyaku_f,channel,cp_dt,kanyu_year,kanyu_month,kanyu_yymm,mob
0,202006,155051921627812,879,大丸松坂屋VISAゴールド,2020-01-06,20200116,0,オンライン,2020-06-01,2020,1,2020年1月,5
1,202006,155051921628414,799,大丸松坂屋VISAゴールド,2020-01-06,20200118,0,オンライン,2020-06-01,2020,1,2020年1月,5


In [295]:
q = """
        SELECT
        cp
        ,channel
        ,kanyu_year
        ,kanyu_month
        ,kanyu_yymm
        ,mob
        ,count(applino) as sum_no_account
        ,sum(case when first_use_dt > 0 then 1 else 0 end) as sum_no_1st_use
        ,sum(case when kaiyaku_f = 1 then 1 else 0 end) as sum_no_kaiyaku
        ,sum(credit_amt_6m) as sum_credit_amt6m
        from 
            df_dup
        group by
            1,2,3,4,5,6
        """

In [296]:
df_sql = sqldf(q, locals())

In [297]:
df_sql.head()

,cp,channel,kanyu_year,kanyu_month,kanyu_yymm,mob,sum_no_account,sum_no_1st_use,sum_no_kaiyaku,sum_credit_amt6m
0,202006,オンライン,2020,1,2020年1月,5,754,334,103,37414
1,202006,オンライン,2020,2,2020年2月,4,1073,330,202,32888
2,202006,オンライン,2020,3,2020年3月,3,402,172,53,19240
3,202006,オンライン,2020,4,2020年4月,2,120,70,5,12172
4,202006,オンライン,2020,5,2020年5月,1,601,376,10,13816


In [298]:
#初回利用率
df_sql['rate_1st_use'] = df_sql['sum_no_1st_use']/df_sql['sum_no_account']

#クレジット初回利用単価
df_sql['unit_1st_amt'] = df_sql['sum_credit_amt6m']/df_sql['sum_no_1st_use']

#解約率
df_sql['rate_kaiyaku'] = df_sql['sum_no_kaiyaku']/df_sql['sum_no_account']


df_sql.head()

,cp,channel,kanyu_year,kanyu_month,kanyu_yymm,mob,sum_no_account,sum_no_1st_use,sum_no_kaiyaku,sum_credit_amt6m,rate_1st_use,unit_1st_amt,rate_kaiyaku
0,202006,オンライン,2020,1,2020年1月,5,754,334,103,37414,0.442971,112.017964,0.136605
1,202006,オンライン,2020,2,2020年2月,4,1073,330,202,32888,0.307549,99.660606,0.188257
2,202006,オンライン,2020,3,2020年3月,3,402,172,53,19240,0.427861,111.860465,0.131841
3,202006,オンライン,2020,4,2020年4月,2,120,70,5,12172,0.583333,173.885714,0.041667
4,202006,オンライン,2020,5,2020年5月,1,601,376,10,13816,0.625624,36.744681,0.016639


CSVアウトプット

In [299]:
df_sql.to_csv(data_dir /r"Output"/f"{output_file}_{input_file}.csv", index=None,header=True,encoding='utf-8-sig')

アフィリエイト別

In [300]:
df_affi = pd.read_excel(data_dir2 /r"input"/f"{input_file2}.xlsx",encoding='shift-jis')
df_affi.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,可否,承認作業月
0,37,げん玉,ポイント,2020-01-08 19:11:08,2020-01-08 19:32:05,155051921643397,BGT,9000,可,1月
1,52,ECナビSP,ポイント,2020-01-08 20:21:44,2020-01-08 20:31:05,155051921643553,BGT,9000,可,1月
2,52,ECナビSP,ポイント,2020-01-08 20:31:50,2020-01-08 20:39:06,155051921643587,BGT,9000,可,1月
3,52,ECナビSP,ポイント,2020-01-08 20:34:03,2020-01-08 20:45:05,155051921643678,BGT,9000,可,1月
4,82,ちょびリッチ。ドットコム,ポイント,2020-01-08 20:32:12,2020-01-08 20:55:06,155051921643744,BGT,9000,可,1月


In [301]:
df_affi = df_affi.rename(columns={'メディア名':'media',
                                  '媒体チャネル':'baitai',
                                  '参照用':'applino'})
df_affi = df_affi[['media','baitai','applino']]
df_affi.head(2)

,media,baitai,applino
0,げん玉,ポイント,155051921643397
1,ECナビSP,ポイント,155051921643553


In [302]:
q = """
        SELECT
        cp
        ,channel
        ,kanyu_year
        ,kanyu_month
        ,kanyu_yymm
        ,mob
        ,media
        ,baitai
        ,count(t1.applino) as sum_no_account
        ,sum(case when first_use_dt > 0 then 1 else 0 end) as sum_no_1st_use
        ,sum(case when kaiyaku_f = 1 then 1 else 0 end) as sum_no_kaiyaku
        ,sum(credit_amt_6m) as sum_credit_amt6m
        from 
            df_dup t1
        inner join
            df_affi t2
        on t1.applino = t2.applino
        group by
            1,2,3,4,5,6,7,8
        """

In [303]:
df_sql = sqldf(q, locals())

In [304]:
df_sql.head()

,cp,channel,kanyu_year,kanyu_month,kanyu_yymm,mob,media,baitai,sum_no_account,sum_no_1st_use,sum_no_kaiyaku,sum_credit_amt6m
0,202006,オンライン,2020,1,2020年1月,5,CLUBPanasonicコイン,ポイント,4,4,0,148
1,202006,オンライン,2020,1,2020年1月,5,ECナビPC,ポイント,7,2,2,16
2,202006,オンライン,2020,1,2020年1月,5,ECナビSP,ポイント,9,4,0,66
3,202006,オンライン,2020,1,2020年1月,5,GetMoney!(JIPC参加企業)（連携用）,ポイント,12,2,2,34
4,202006,オンライン,2020,1,2020年1月,5,NetMile（すぐたま）,ポイント,13,6,4,168


In [305]:
df_sql.shape[0]

152

In [306]:
df_sql.to_csv(data_dir /r"Output"/f"{output_file2}_{input_file}.csv", index=None,header=True,encoding='utf-8-sig')